# PRiSM Tool

In the "Cell" menu, select "Run All".

To re-run from scratch, click the ⏩ button.

Julia uses "Just Ahead of Time" (JAOT) (aka "Just in Time" (JIT)) compilation, the first time you run this notebook will be substantially slower than subsequent runs, particularly as plotting is concerned.

## Parameters
Some of the more "twiddle-able" parameters are collected here.

In [ ]:
#image_name = "images/prism_square.png"
#image_name = "images/scribble.png"
image_name = "images/scribble2.png"

max_level = 7;

## Activate Package
This takes care of all the dependencies (other pieces of software) that we rely on.
The first time you run it, this will require an internet connection and may take a long time!

In [ ]:
import Pkg; Pkg.activate("."); Pkg.instantiate()

In [ ]:
using Revise  # Development tool - must come first

## Custom Mesh Package
This package provides a simple wrapper for some of the functionality of [p4est](https://p4est.org), a powerful, low-level library for massively-parallel adaptive mesh refinement (AMR). If/when this stabilizes, it'll be made available as an indepedent package.

In [ ]:
using SimpleTreeMeshes

## Helpers
Other packages to help with plotting, printing, etc.

In [ ]:
using Plots
using Printf
using Images, FileIO

## An image used to drive the simulation
If you want to refine a lot, make sure it's high-enough resolution!

In [ ]:
img = Gray.(load(image_name))
sz = size(img)
mask_size = min(sz[1], sz[2])
img2 = img[1:mask_size, 1:mask_size]
mask_r = convert(Array{Float16}, img2)
mask = zeros(size(mask_r))
for i = 1:mask_size
    for j = 1:mask_size
        mask[i,j] = mask_r[mask_size - j + 1,i]
    end
end
display(img2)

## Defining a Physical Problem

In [ ]:
struct Problem
  name::String
  boundary_conditions::StokesBoundaryConditions
  eta::Function
  eta_min::Float64
  eta_max::Float64
  eta_ref::Float64
  f_x::Function
  f_y::Function
  f_p::Function
  vx_ref::Function
  vy_ref::Function
  p_ref::Function
  coordinate_scale::Float64
  coordinate_offset_x::Float64
  coordinate_offset_y::Float64
end

In [ ]:
function image_value(x, y)
  @assert 0 <= x <= 1  
  @assert 0 <= y <= 1  
  return mask[Int(floor(x * mask_size)), Int(floor(y * mask_size))]
end;

function problem_Image()
    eta(x, y) = 1.0
    eta_min = 1.0
    eta_max= 1.0
    eta_ref = sqrt(eta_min * eta_max)

    f_vy(x,y) = -10.0 * (1.0 - image_value(x,y))
    f_vx(x, y) = 0.0
    f_p(x, y) = 0.0 
    
    vx_ref(x, y) = 0.0
    vy_ref(x, y) = 0.0
    p_ref(x, y) = 0.0
 
    return Problem("Image", stokes_boundary_free_slip, eta, eta_min, eta_max, eta_ref, f_vx, f_vy, f_p, 
                   vx_ref, vy_ref, p_ref, 1.0, 0.0, 0.0)
    
end;

## Refinement function

In [ ]:
min_level = 1;

function refinement_function(x, y, d, level)::Bool
    
    if level <= min_level;  return true; end
    if level >= max_level;  return false;  end

    # attempt to refine if all local pixels aren't in a constant band of values 
    mx_min = max(1, Int(ceil(x * (mask_size - 1))))
    my_min = max(1, Int(ceil(y * (mask_size - 1))))

    mx_max = max(1, Int(ceil((x + d) * (mask_size - 1))))
    my_max = max(1, Int(ceil((y + d) * (mask_size - 1))))

    max_local = 0
    min_local = 1.0

    for i = mx_min:mx_max
        for j = my_min:my_max
            val = mask[i,j]
            if val > max_local
                max_local = val
            end
            if val < min_local
                min_local = val
            end
        end
    end

    return max_local - min_local > 0.2
end;

## Create our tree object

In [ ]:
start = time()
tree = CreateSimpleTreeMesh(refinement_function);
@printf "⏲️ %2.2f s\n" time()- start

## Plot to see the grid

In [ ]:
start = time()
plot_size = 600
p_grid = plot(axis=nothing, xaxis=false, yaxis=false, size=(plot_size, plot_size), aspect_ratio=1)
plot_grid!(p_grid, tree, RGB(0.5, 0.5, 0.5))
plot_face_numbers!(p_grid, tree)
plot_element_numbers!(p_grid, tree, offset=tree.nf) # Number after faces, as in our Stokes system
plot_corner_numbers!(p_grid, tree) # A separate numbering system
display(p_grid)
@printf "⏲️ %2.2f s\n" time()- start

## Solve the Stokes system

In [ ]:
start = time()
tree = CreateSimpleTreeMesh(refinement_function)
@printf "⚙️ %d elements, %d faces, %d corners " tree.ne tree.nf tree.nc
@printf "⏲️ %2.2f s\n" time() - start

In [ ]:
start = time()
h_min = 2.0^(-(max_level))
problem = problem_Image()
A, b, Kcont = assemble_system(tree, problem, h_min)
x = A\b
v, p = sol2vp(tree, x, Kcont)
@printf "⏲️ %2.2f s\n" time() - start

In [ ]:
start = time()
plot_size = 800
p_stokes = plot(axis=nothing, xaxis=false, yaxis=false, size=(plot_size, plot_size), aspect_ratio=1)
plot_element_field!(p_stokes, tree, p)
c = RGB(0.25, 0.25, 0.25)
plot_grid!(p_stokes, tree, c)
plot_averaged_velocity_field!(p_stokes, tree, v, color=c)
display(p_stokes)
@printf "⏲️ %2.2f s\n" time() - start

## Push particles and generate traces

In [ ]:
struct Trace
  id::Int # An integer identifier
  x::Array{Float64}  
  y::Array{Float64}  
  t::Array{Float64}
  p::Array{Float64} 
end;

In [ ]:
function create_trace(tree::SimpleTreeMesh, x0, y0, v, p, dt=300.0, nsteps=200; id=0)
    @assert length(v) == tree.nf
    @assert length(p) == tree.ne
    trace = Trace(id, [], [], [], [])
    e_id = tree.ne ÷ 2
    particle_y = y0
    particle_x = x0
    t = 0
    dt = 1.0
    for step in 1:nsteps
    
       # Collect data from the current position
       append!(trace.x, particle_x)
       append!(trace.y, particle_y)
       append!(trace.t, t)
       e_id = locate_point(tree, particle_x, particle_y, e_id)
       append!(trace.p, p[e_id]) 
        
       # An extremely naive (forward Euler) particle position update
       vx, vy = get_point_velocity(tree, particle_x, particle_y, e_id, v)
       particle_x += dt * vx
       particle_y += dt * vy
        
       if particle_x < tree.coordinate_offset_x; particle_x = tree.coordinate_offset_x; end
       if particle_x > tree.coordinate_offset_x + tree.coordinate_scale; particle_x = tree.coordinate_offset_x + tree.coordinate_scale; end
       if particle_y < tree.coordinate_offset_y; particle_y = tree.coordinate_offset_y; end
       if particle_y > tree.coordinate_offset_y + tree.coordinate_scale; particle_y = tree.coordinate_offset_y + tree.coordinate_scale; end
       t += dt
    end
    return trace
end;

In [ ]:
start = time()
traces = []
x0s = LinRange(0.1, 0.9, 7)
for i = 1:length(x0s)
  x0 = x0s[i] 
  y0 = x0
  append!(traces, [create_trace(tree, x0, y0, v, p, id=i)])
end
@printf "⏲️ %2.2f s\n" time() - start

In [ ]:
start = time()
# Plot on the same plot as above and re-display
for trace in traces
    plot!(p_stokes, trace.x, trace.y, markershape=:circle, label=nothing, color=trace.id)
    plot!(p_stokes, [trace.x[1]], [trace.y[1]], markershape=:star, markersize=10, label=nothing, color=trace.id)
end
display(p_stokes)
@printf "⏲️ %2.2f s\n" time() - start

In [ ]:
start = time()
p_traces = plot(xlabel="t", ylabel="p", size=(800,600))
for trace in traces
    label = @sprintf "Trace %d" trace.id
    plot!(p_traces, trace.t, trace.p, marker=:circle, label=label, color=trace.id)
    plot!(p_traces, [trace.t[1]], [trace.p[1]], markershape=:star, markersize=10, label=nothing, color=trace.id)
end
display(p_traces)
@printf "⏲️ %2.2f s\n" time() - start

## Write traces to files

In [ ]:
start = time()
if ~isdir("traces"); mkdir("traces"); end
for (trace_id, trace) in enumerate(traces)
    filename = @sprintf "traces/trace_%04d.txt" trace_id
    open(filename, "w") do io
        for i = 1:length(trace.t)
            line = @sprintf "%g %g\n" trace.t[i] trace.p[i]
            write(io, line)
        end
    end
end;
@printf "⏲️ %2.2f s\n" time() - start